In [1]:
import random
import cnn_builder as cbuild
import config
import copy
import glob
import importlib
import lipiodol_methods as lm
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import niftiutils.masks as masks
import numpy as np
import os
from os.path import *
import shutil
from scipy.ndimage.morphology import binary_closing, binary_opening, binary_dilation
from skimage.morphology import ball, label

from keras import backend as K
from keras.engine import Input, Model
from keras.layers import *
from keras.optimizers import Adam
from keras.utils import np_utils
%matplotlib inline

C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
importlib.reload(config)
C = config.Config()

In [3]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

In [92]:
seg = glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids"))[0]

In [ ]:
lesion_id = "07"
glob.glob(join(target_dir,lesion_id,"CT24h","*"))

In [27]:
for f in glob.glob(join(target_dir,lesion_id,"CT24h","*")):
    if "5 0" not in f:
        shutil.rmtree(f)
    else:
        shutil.move()

# MRI

### Data prep

In [4]:
# MRI
for seg in glob.glob(join(target_dir,"*","masks","liver bl.ids")):
    lesion_id = dirname(seg)[len(target_dir)+1:-len("masks")-1]
    if exists(join(C.full_img_dir, lesion_id+"_mri_art.npy")):
        continue
    
    mri_img, mri_dims = hf.nii_load(join(target_dir, lesion_id, "MRI-BL", "t1-art.nii.gz"), True, True)
    mask, _ = masks.get_mask(seg, mri_dims, mri_img.shape)

    mri_img -= mri_img.min()
    mri_img /= mri_img.std()
    
    np.save(join(C.full_img_dir, lesion_id+"_mri_art.npy"), mri_img)
    np.save(join(C.full_img_dir, lesion_id+"_mr_bl_liver_mask.npy"), mask > 128)
    
    #mri_img, mri_dims = hf.nii_load(join(target_dir, lesion_id, "MRI-BL", "t1-pre.nii.gz"), True, True)
    #mri_img -= mri_img.min()
    #mri_img /= mri_img.std()
    #np.save(join(C.full_img_dir, lesion_id+"_mri_art.npy"), mri_img)

In [13]:
data_dir = "D:\\JHU\\JHU HCC"

for path in glob.glob(join(data_dir,"*","*.nii")):
    lesion_id = basename(dirname(path))
    lesion_id = lesion_id[:lesion_id.find(" ")]
    
    mri_img, mri_dims = hf.nii_load(path)
    
    mask_paths = glob.glob(join(dirname(path),"*Segs","Liver","*.ids"))
    if len(mask_paths) != 1:
        print(path)
        continue
    mask, _ = masks.get_mask(mask_paths[0], mri_dims, mri_img.shape)

    mri_img -= mri_img.min()
    mri_img /= mri_img.std()
    
    np.save(join(C.full_img_dir, lesion_id+"_mri_art.npy"), mri_img)
    np.save(join(C.full_img_dir, lesion_id+"_mr_bl_liver_mask.npy"), mask > 128)

### Train model

In [ ]:
importlib.reload(cbuild)
model = cbuild.build_unet()
#model.summary(120)

In [ ]:
hist = model.fit_generator(cbuild.train_gen_mri(), epochs=500, steps_per_epoch=5)

In [20]:
model.save(join(C.model_dir, "mri_liver.hdf5"))

### Test

In [72]:
importlib.reload(lm)

<module 'lipiodol_methods' from 'C:\\Users\\Clinton\\Documents\\Lipiodol\\lipiodol_methods.py'>

In [81]:
lesion_id = "12"
mri_nii_path = join(target_dir, lesion_id, "MRI-30d", "t1-art.nii.gz")
save_path = join(target_dir, lesion_id, "masks", "mri_30d_liver")
tumor_mask_path = join(target_dir, lesion_id, "masks", "tumor_30dFU_MRI")
lm.seg_liver_mri_from_path(mri_nii_path, save_path, model, tumor_mask_path)

# CT

In [ ]:
train_set = []
for seg in glob.glob(join(target_dir,"*","masks","wholeliver_24hCT*.ids")):
    lesion_id = dirname(seg)[len(target_dir)+1:-len("masks")-1]
    if exists(join(C.full_img_dir, lesion_id+"_ct.npy")):
        #train_set.append([np.load(join(C.full_img_dir, lesion_id+"_ct.npy")),
        #                  np.load(join(C.full_img_dir, lesion_id+"_mask.npy")),
        #                  lesion_id])
        continue
    
    ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
    mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)

    ct_img = tr.apply_window(ct_img)
    ct_img -= np.amin(ct_img)
    
    np.save(join(C.full_img_dir, lesion_id+"_ct.npy"), ct_img)
    np.save(join(C.full_img_dir, lesion_id+"_mask.npy"), mask > 128)

In [91]:
importlib.reload(cbuild)
model = cbuild.build_ct_unet()
model.summary(120)

<module 'cnn_builder' from 'C:\\Users\\Clinton\\Documents\\Lipiodol\\cnn_builder.py'>

In [ ]:
hist = model.fit_generator(cbuild.train_gen(), epochs=100, steps_per_epoch=50)

In [38]:
model.save(join(C.model_dir, "ct17.hdf5"))

In [107]:
lesion_id = "18"
ct_img, ct_dims = hf.dcm_load(join(target_dir, lesion_id, "CT24h"), True, True)
lm.seg_liver_ct(ct_img, join(target_dir, lesion_id, "ct24_liver"), ct_dims, model)

# Misc

In [12]:
info = {}

for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        
    seg_liver(ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims)
    break